# Development and test of the PyironFlow Gui

In [1]:
%config IPCompleter.evaluation='unsafe'

import sys
from pathlib import Path
sys.path.insert(0, str(Path(Path.cwd()).parent) + '/pyiron_nodes')

In [2]:
!npx esbuild js/widget.jsx --minify --format=esm --bundle --outdir=static

⠙
  static/widget.js    1.7mb ⚠️
  static/widget.css  15.1kb

⚡ Done in 151ms
⠙

In [3]:
from python.pyironflow import PyironFlow

pf = PyironFlow([])
pf.gui

In [5]:
from pyiron_workflow import Workflow
from python.reactflow import get_import_path

In [7]:
wf = Workflow('phonon')

import pyiron_nodes.atomistic as atomistic
# Workflow.register("node_library.atomistic", "atomistic") 

wf.bulk = atomistic.structure.build.Bulk('Al', cubic=True)
wf.phonopy = atomistic.property.phonons.CreatePhonopy()

In [27]:
get_import_path(wf.bulk)

'node_library.atomistic.structure.build.bulk'

In [38]:
'__name__' in dir(wf.phonopy)

False

In [37]:
hasattr(wf.phonopy, '__name__')

AmbiguousOutputError: Tried to access the channel value of phonopy, but this is only possible when there is a single output channel -- phonopy has: ['phonopy', 'out']. Access probably occurred attempting to use this object like an output channel, e.g. with injection or to form a connection. Either make sure it has exactly one output channel, or use the particular channel you want directly.

In [33]:
wf.phonopy.__class__.__name__

'create_phonopy'

In [28]:
get_import_path(wf.phonopy)

AmbiguousOutputError: Tried to access the channel value of phonopy, but this is only possible when there is a single output channel -- phonopy has: ['phonopy', 'out']. Access probably occurred attempting to use this object like an output channel, e.g. with injection or to form a connection. Either make sure it has exactly one output channel, or use the particular channel you want directly.

In [20]:
get_import_path(wf.phonopy)

AmbiguousOutputError: Tried to access the channel value of phonopy, but this is only possible when there is a single output channel -- phonopy has: ['phonopy', 'out']. Access probably occurred attempting to use this object like an output channel, e.g. with injection or to form a connection. Either make sure it has exactly one output channel, or use the particular channel you want directly.

In [15]:
from phonopy.api_phonopy import Phonopy

from structuretoolkit.common import atoms_to_phonopy
import pyiron_workflow

structure = wf.bulk.run()
phonopy = Phonopy(unitcell=atoms_to_phonopy(structure))
phonopy

In [21]:
# wf.phonopy.pull()

## Concepts to convert dataclass into node

In [ ]:
# json.loads(pf.wf_widgets[0].gui.nodes)[1];

from python.reactflow import get_node_types, _get_type_name, _get_generic_type
import typing

def get_node_types(node_io):
    node_io_types = list()
    for k in node_io.channel_dict:
        type_hint = node_io[k].type_hint
        # print (k, type(type_hint))
        if isinstance(type_hint, typing._UnionGenericAlias):
            # print (k, type_hint)
            type_hint = _get_generic_type(type_hint)

        node_io_types.append(_get_type_name(type_hint))
    return node_io_types

# wf.inp_calc_md.inputs.channel_dict['temperature'].type_hint

In [ ]:
get_node_types(wf.bulk.inputs), get_node_types(wf.inp_calc_md.inputs);

In [ ]:
wf.bulk.inputs.channel_dict['vacancy_index'].type_hint

In [ ]:
def f(a:int=3):
    return a

In [ ]:
f.__annotations__

In [ ]:
def f(a, b, c: int=3):
    return a

In [ ]:
from inspect import signature

In [ ]:
f'f{str(signature(f))}'

In [ ]:
a = pn.atomistic.structure.build.CubicBulkCell

In [ ]:
signature(a)

In [ ]:
pf = PyironFlow([])
pf.gui

In [ ]:
wf = pf.get_workflow()
wf

In [ ]:
pf = PyironFlow([wf])
pf.gui

In [ ]:
from python.reactflow import get_node_types, _get_type_name, _get_generic_type
import typing

def get_node_types(node_io):
    node_io_types = list()
    for k in node_io.channel_dict:
        type_hint = node_io[k].type_hint
        # print (k, type(type_hint))
        if isinstance(type_hint, typing._UnionGenericAlias):
            # print (k, type_hint)
            type_hint = _get_generic_type(type_hint)

        node_io_types.append(_get_type_name(type_hint))
    return node_io_types

In [ ]:
from pyiron_workflow import as_dataclass_node
from typing import Optional
from dataclasses import dataclass

# @as_dataclass_node
@dataclass
class InputCalcMD:
    temperature: Optional[int | float] = 300
    n_ionic_steps: int = 10_000
    n_print: int = 100
    pressure: Optional[int | float] = None
    time_step: Optional[int | float] = 1.0
    temperature_damping_timescale: Optional[int | float] = 100.0
    pressure_damping_timescale: Optional[int | float] = 1000.0
    seed: Optional[int] = None
    tloop: Optional[float] = None
    initial_temperature: Optional[float] = None
    langevin: bool = False
    delta_temp: Optional[float] = None
    delta_press: Optional[float] = None

# InputCalcMD()

In [ ]:
ic_md = InputCalcMD()
get_node_types(ic_md.inputs)

In [ ]:
from pyiron_nodes.atomistic.calculator.data import InputCalcMD

ic_md = as_dataclass_node(InputCalcMD)()
get_node_types(ic_md.inputs)

In [ ]:
InputCalcMD??

In [ ]:
pn.atomistic.engine.lammps.SetInputCalcMD().color

In [ ]:
type(pn.atomistic.calculator.data.InputCalcMD().run())

In [ ]:
isinstance(pn.atomistic.calculator.data.InputCalcMD().run(), pyiron_nodes.atomistic.calculator.data.InputCalcMD.dataclass)

In [ ]:
def dataclass_to_function(instance: Any) -> Callable:
    dataclass_type = type(instance)
    fields = dataclasses.fields(instance)
    
    # Create argument list and function code
    arg_list_str = ', '.join([f'{f.name}={repr(getattr(instance, f.name))}' for f in fields])
    function_code = f'def {dataclass_type.__name__}_func({arg_list_str}):\n' \
                    f'    return {dataclass_type.__name__}({arg_list_str})'
    
    print(function_code)  # This will print the function code

    # Create function in local context
    namespace = {dataclass_type.__name__: dataclass_type}
    exec(function_code, namespace)
    
    # Populate func with the newly created function and return
    func = namespace[f'{dataclass_type.__name__}_func']
    return func


@dataclasses.dataclass
class InputCalcMD:
    temperature: float = 300
    pressure: float = 0

fn = dataclass_to_function(InputCalcMD())
print(inspect.getsource(fn))



In [ ]:
@dataclasses.dataclass
class InputCalcMD:
    temperature: float = 300
    pressure: float = 0

fn = dataclass_to_function(InputCalcMD())
fn

In [ ]:
Workflow.wrap.as_function_node(fn)


In [ ]:
import dataclasses
import dill
from typing import Any

def dataclass_to_function(instance: Any):
    dataclass_type = type(instance)
    fields = dataclasses.fields(instance)
    
    # Create argument list and function code
    arg_list_str = ', '.join([f'{f.name} = {repr(getattr(instance, f.name))}' for f in fields])
    function_code = f'def {dataclass_type.__name__}_func({arg_list_str}):\n    return {dataclass_type.__name__}({arg_list_str})'
      
    # Create function in local context  
    namespace = {dataclass_type.__name__: dataclass_type}
    exec(function_code, namespace)
    func = namespace[dataclass_type.__name__ + '_func']
    
    # Use dill's dumps to serialize the function and then loads to get function object
    serialized_func = dill.dumps(func)
    loaded_func = dill.loads(serialized_func)
    
    return loaded_func

@dataclasses.dataclass
class MyDataclass:
    temperature: float = 300
    pressure: float = 0

input_instance = MyDataclass()

fn = dataclass_to_function(input_instance)

# Now dill.source.getsource should work, but it will return the serialized byte representation, not the original Python code.
print(dill.source.getsource(fn))  
